In [1]:
from quchem.Hamiltonian_Generator_Functions import *
from quchem.Graph import *
### HAMILTONIAN start
Molecule = 'LiH'
geometry = None # [('H', (0., 0., 0.)), ('H', (0., 0., 0.74))]
basis = 'sto-6g'


### Get Hamiltonian
Hamilt = Hamiltonian_PySCF(Molecule,
                     run_scf=1, run_mp2=1, run_cisd=1, run_ccsd=1, run_fci=1,
                     basis=basis,
                     multiplicity=1,
                     geometry=geometry)  # normally None!
QubitHamiltonian = Hamilt.Get_Qubit_Hamiltonian(threshold=None, transformation='BK')
### HAMILTONIAN end
QubitHamiltonian

(-3.989972418189361+0j) [] +
(0.04037460325642971+0j) [X0 X1 X2] +
(-0.0031615876042055736+0j) [X0 X1 X2 X3 Y7 Y11] +
(0.001620476926375608+0j) [X0 X1 X2 Y3 Y5] +
(0.0036508502556758934+0j) [X0 X1 X2 Z3] +
(0.0018998726581460853+0j) [X0 X1 Z2 X3 Y7 Z9 Y10 X11] +
(0.0046952605402461375+0j) [X0 X1 Z2 Y3 Y4 X5] +
(-0.0012113027134832408+0j) [X0 X1 X3 X4 Y7 Y11] +
(0.0010441957704833429+0j) [X0 X1 X3 Y4 Y5 Z6 Z7] +
(0.0012113027134832408+0j) [X0 X1 X3 Y4 Z5 Y7 Z9 Z10 X11] +
(0.002559439110553769+0j) [X0 X1 X3 Z4 Y5 Y6 Z7] +
(6.648581206866472e-05+0j) [X0 X1 X3 Z4 Z5 Y7 Z9 Y10 X11] +
(0.0012777885255519056+0j) [X0 X1 X3 Z4 Y7 Z9 Y10 X11] +
(0.0009618499558327647+0j) [X0 X1 X3 X6 Y7 Y11] +
(-0.0001567017221040011+0j) [X0 X1 X3 Z6 Y7 Z9 Y10 X11] +
(0.0009618499558327647+0j) [X0 X1 X3 Y7 X8 Y11] +
(-0.0009618499558327647+0j) [X0 X1 X3 Y7 Y8 Z10 X11] +
(-0.0001567017221040011+0j) [X0 X1 X3 Y7 Z8 Z9 Y10 X11] +
(0.0008051482337287634+0j) [X0 X1 X3 Y7 Z8 Y10 X11] +
(0.015294910119291495+0j) [X0 X1

The LiH ground state under JW is:

$$|\psi \rangle_{HF}^{ground} = | 1 1 1 1 \:  0 0 0 0 0 0 0 0 \rangle$$

and under BK is:

$$|\psi \rangle_{HF}^{ground} = | 1 0 1 0 0 0 0 0 0 0 0 0 \rangle$$

Use NOON to reduces this:

In [2]:
from quchem.Ansatz_Generator_Functions import *

##
NOON_spins_combined, NMO_basis = Hamilt.Get_NOON()
##
Hamilt.Get_CCSD_Amplitudes()

NOON_spins_combined

array([1.99991562e+00, 1.96764864e+00, 2.71791037e-02, 7.62575442e-05,
       2.59018655e-03, 2.59018655e-03])

LOOK at PHYS. REV. X, **8**, 031022 (2018)

Due to NOON:
- indices 0,1 assumed to ALWAYS be filled
- indices 8,9 and 10,11 ALWAYS unfilled! <--- decision



# Therefore Simplify Hamiltonian

In [3]:
ansatz_obj = BK_Qubit_Reduction(QubitHamiltonian,
                             Hamilt.molecule.n_electrons,
                             Hamilt.molecule.n_qubits)

The LiH ground state under JW is:

$$|\psi \rangle_{HF}^{ground} = | 1 1 1 1 \:  0 0 0 0 0 0 0 0 \rangle$$

and under BK is:

$$|\psi \rangle_{HF}^{ground} = | 1 0 1 0 0 0 0 0 0 0 0 0 \rangle$$


we have **fixed occupations** of indices [0,1,8,9,10,11]

- therefore can remove these terms from the Hamiltonian!

In [4]:
list_of_qubit_indices_to_remove = [0,1,8,9,10,11]
reduced_Qubit_Hamiltonian = ansatz_obj.Remove_indices_from_Hamiltonian(list_of_qubit_indices_to_remove)
print('Hamiltonian size reduced: {} --> {}'.format(len(list(QubitHamiltonian)), len(list(reduced_Qubit_Hamiltonian))))

Hamiltonian size reduced: 631 --> 239


In [5]:
qubit_re_label_dict, reduced_RE_LABELLED_Qubit_Hamiltonian = ansatz_obj.Re_label_Hamiltonian(reduced_Qubit_Hamiltonian)
reduced_RE_LABELLED_Qubit_Hamiltonian

(-7.2648318379716885+0j) [] +
(0.10945352587994771+0j) [X0] +
(0.009344683608047986+0j) [X0 X1 X2 X3] +
(-0.001062048701644394+0j) [X0 X1 X2 Y5] +
(0.001620476926375608+0j) [X0 X1 Y2 X3] +
(-0.00027634696897384627+0j) [X0 X1 Y2 Y3 Z4 Z5] +
(0.001062048701644394+0j) [X0 X1 Y2 Z3 Y5] +
(-0.001620476926375608+0j) [X0 X1 Z2 X3] +
(0.00483644372800671+0j) [X0 X1 Z2 Y3 Y4 Z5] +
(-0.003648349033603868+0j) [X0 X1 Z2 Z3 Y5] +
(-0.002586300331959474+0j) [X0 X1 Z2 Y5] +
(0.0040759180851561785+0j) [X0 X1 X4 Y5] +
(-0.009750569335355262+0j) [X0 X1 Z4 Y5] +
(-0.023380157572586935+0j) [X0 X1 Y5] +
(-0.001620476926375608+0j) [X0 Y1 X2 X3] +
(0.0003218455426223995+0j) [X0 Y1 X2 Y3] +
(0.004739070747201672+0j) [X0 Y1 Y2 X3] +
(0.004560096759032863+0j) [X0 Y1 Y2 X3 Z4] +
(-0.0058630615794319895+0j) [X0 Y1 Z2 X3] +
(0.0002415642481650689+0j) [X0 Y1 Y3] +
(-0.00483644372800671+0j) [X0 Y1 Y3 X4] +
(-0.0040759180851561785+0j) [X0 Y1 Z3 Y4 X5] +
(-0.005674651250199083+0j) [X0 Y1 Z3 Z4 X5] +
(0.040548940098522

In [ ]:
new_Molecular_H_MATRIX = Hamilt.Get_sparse_Qubit_Hamiltonian_matrix(reduced_RE_LABELLED_Qubit_Hamiltonian)


from scipy.sparse.linalg import eigs
eig_values, eig_vectors = eigs(new_Molecular_H_MATRIX)
new_FCI_Energy = min(eig_values)


print('new_FCI = ', new_FCI_Energy, 'VS old FCI:', Hamilt.molecule.fci_energy)

## Looking at Ansatz

In [ ]:
# automate:
# reduced_Sec_Quant_CC_ops_ia, reduced_Sec_Quant_CC_ops_ijab, reduced_theta_parameters_ia, reduced_theta_parameters_ijab =ansatz_obj.Remove_NOON_terms(
#     NOON=NOON_spins_combined,
#     occ_threshold= 1.999,
#     unocc_threshold=1e-4,
#     indices_to_remove_list_manual=None, 
#     single_cc_amplitudes=Hamilt.molecule.single_cc_amplitudes,
#     double_cc_amplitudes=Hamilt.molecule.double_cc_amplitudes,
#     singles_hamiltonian=Hamilt.singles_hamiltonian,
#     doubles_hamiltonian=Hamilt.doubles_hamiltonian,
#     tol_filter_small_terms=None)

# manual
reduced_Sec_Quant_CC_ops_ia, reduced_Sec_Quant_CC_ops_ijab, reduced_theta_parameters_ia, reduced_theta_parameters_ijab =ansatz_obj.Remove_NOON_terms(
    NOON=NOON_spins_combined,
    indices_to_remove_list_manual=[0,1,8,9,10,11])

In [ ]:
reduced_Sec_Quant_CC_ops_ijab

In [ ]:
ia_terms, ijab_terms, ia_theta, ijab_theta = ansatz_obj.Get_ia_and_ijab_terms()
print('REDUCTION')
print('ia_terms', len(ia_terms), 'TO', len(reduced_Sec_Quant_CC_ops_ia))
print('ijab_terms', len(ijab_terms), 'TO', len(reduced_Sec_Quant_CC_ops_ijab))

In [ ]:
Qubit_Op_list_Second_Quant_CC_Ops_ia, Qubit_Op_list_Second_Quant_CC_Ops_ijab = ansatz_obj.UCCSD_single_trotter_step(reduced_Sec_Quant_CC_ops_ia,
                                                                                                                    reduced_Sec_Quant_CC_ops_ijab,
                                                                                                                     transformation='BK')

# # NO reduction!
# Qubit_Op_list_Second_Quant_CC_Ops_ia, Qubit_Op_list_Second_Quant_CC_Ops_ijab = ansatz_obj.UCCSD_single_trotter_step(ia_terms,
#                                                                                                                     ijab_terms
#                                                                                                                     transformation='BK')


In [ ]:
print(Qubit_Op_list_Second_Quant_CC_Ops_ijab)
print('')

print('only considering')
print(list(Qubit_Op_list_Second_Quant_CC_Ops_ijab[0])[0])
print(list(Qubit_Op_list_Second_Quant_CC_Ops_ijab[2])[0])

In [ ]:
from openfermion.ops import QubitOperator

op_1 = QubitOperator(list(list(Qubit_Op_list_Second_Quant_CC_Ops_ijab[0])[0].terms.items())[0][0], -1j)
op_2 = QubitOperator(list(list(Qubit_Op_list_Second_Quant_CC_Ops_ijab[2])[0].terms.items())[0][0], -1j)

Qubit_Op_list_Second_Quant_CC_Ops_ijab_SELECTION = [op_1, op_2]
print(Qubit_Op_list_Second_Quant_CC_Ops_ijab_SELECTION)

theta_ijab_selected = [0 for _ in range(len(Qubit_Op_list_Second_Quant_CC_Ops_ijab_SELECTION))] 
theta_ijab_selected

In [ ]:
reduced_CC_ijab = ansatz_obj.Remove_indices_from_CC_qubit_operators(Qubit_Op_list_Second_Quant_CC_Ops_ijab_SELECTION, list_of_qubit_indices_to_remove)
reduced_CC_ijab

In [ ]:
reduced_RE_LABELLED_CC_ijab = ansatz_obj.Re_label_CC_qubit_operators( qubit_re_label_dict, 
                                                                   reduced_CC_ijab)
reduced_RE_LABELLED_CC_ijab

## HF input state

In [ ]:
print('old input = ', ansatz_obj.Get_BK_HF_state_in_OCC_basis())

print('BUT following indices removed:', list_of_qubit_indices_to_remove)

ansatz_obj.New_BK_HF_state(list_of_qubit_indices_to_remove)

# Ansatz

look at: From PHYS. REV. X, **8**, 031022 (2018)

can make a further reduction!

In [ ]:
reduced_RE_LABELLED_CC_ijab

In [ ]:
import random
# theta_ia_random_input = [random.uniform(0, 2*np.pi) for _ in range(len(reduced_RE_LABELLED_CC_ia))]
theta_ijab_random_input = [random.uniform(0, 2*np.pi) for _ in range(len(reduced_RE_LABELLED_CC_ijab))]

In [ ]:
input_state = ansatz_obj.New_BK_HF_state(list_of_qubit_indices_to_remove)
print('input_state =', input_state)

UCCSD_ansatz_Q_Circ_obj = Ansatz_Circuit(input_state,
                                     None,                                     # <-- note no ia terms!
                                     reduced_RE_LABELLED_CC_ijab)

ansatz_cirq_circuit =UCCSD_ansatz_Q_Circ_obj.Get_Full_HF_UCCSD_QC(
                                         Theta_param_list_ijab=theta_ijab_random_input,
                                         ia_first=True)
ansatz_cirq_circuit

# Running Example

In [ ]:
Hamiltonian_graph_obj = Openfermion_Hamiltonian_Graph(reduced_RE_LABELLED_Qubit_Hamiltonian)

commutativity_flag = 'AC' ## <- defines relationship between sets!!!
plot_graph = False
Graph_colouring_strategy='largest_first'
anti_commuting_sets = Hamiltonian_graph_obj.Get_Clique_Cover_as_QubitOp(commutativity_flag, Graph_colouring_strategy=Graph_colouring_strategy, plot_graph=plot_graph)
anti_commuting_sets

In [ ]:
from quchem.LCU_method import *

In [ ]:
N_QUBITS = Hamilt.molecule.n_qubits - len(list_of_qubit_indices_to_remove)
n_shots= 100

def GIVE_ENERGY_LCU(theta_ijab):
    
    ansatz_cirq_circuit =UCCSD_ansatz_Q_Circ_obj.Get_Full_HF_UCCSD_QC(
                                        Theta_param_list_ia=None, 
                                         Theta_param_list_ijab=theta_ijab,
                                         ia_first=True)

    VQE_exp_LCU =  VQE_Experiment_LCU_UP(anti_commuting_sets,
                     ansatz_cirq_circuit,
                     n_shots,
                     N_QUBITS,
                     N_indices_dict=None)#{7:0, 8:1, 9:0, 10:1})
    
    return VQE_exp_LCU.Calc_Energy().real


In [ ]:
import random
theta_ijab_random_input = [random.uniform(0, 2*np.pi) for _ in range(len(reduced_RE_LABELLED_CC_ijab))]

print(theta_ijab_random_input)

GIVE_ENERGY_LCU(theta_ijab_random_input)

## Simulating

In [ ]:
def calc_gradient_ADAM(theta_ia_theta_jab_list):
    
    grad_list=[]
    
    for index, theta in enumerate(theta_ia_theta_jab_list):
        
        new_theta_list = theta_ia_theta_jab_list.copy()
        new_theta_list[index] = theta + np.pi/4
        
        Obs_PLUS = GIVE_ENERGY_LCU(new_theta_list)
               
        
        new_theta_list[index] = theta - np.pi/4
        Obs_MINUS = GIVE_ENERGY_LCU(new_theta_list)
        
        gradient = Obs_PLUS - Obs_MINUS
        
        grad_list.append(gradient)
        

    return np.array(grad_list)

In [ ]:
### optimizer
from quchem.Scipy_Optimizer import _minimize_Adam, Optimizer

In [ ]:
custom_optimizer_DICT = {'learning_rate': 0.1, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-8,
                           'delta': 1e-8, 'maxfev': 15000}
tol = 1e-6

GG = Optimizer(GIVE_ENERGY_LCU,
                   theta_ijab_random_input,
                   args=(),
                   method=_minimize_Adam,
                   jac=calc_gradient_ADAM,
                   hess=None,
                   hessp=None,
                      bounds=None,
                   constraints=None,
                   tol=tol, 
                   display_convergence_message=True,
                   display_steps=True, 
                   custom_optimizer_DICT=custom_optimizer_DICT)
GG.get_env(100)
GG.plot_convergence()
plt.show()

In [ ]:
# Nelder-Mead

GG = Optimizer(GIVE_ENERGY_LCU,
                   theta_ijab_random_input,
                   args=(),
                   method='Nelder-Mead',
                   jac=None,
                   hess=None,
                   hessp=None,
                      bounds=None,
                   constraints=None,
                   display_convergence_message=True,
                   display_steps=True)

GG.get_env(50)
GG.plot_convergence()
plt.show()

In [ ]:
# # finite difference

# custom_optimizer_DICT = {'learning_rate': 0.1, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-8,
#                            'delta': 1e-8, 'maxfev': 15000}

# GG = Optimizer(GIVE_ENERGY_LCU,
#                    theta_combined_random_input,
#                    args=(),
#                    method=_minimize_Adam,
#                    jac=None,                 # <-- finite difference approach!
#                    hess=None,
#                    hessp=None,
#                       bounds=None,
#                    constraints=None,
#                    tol=1e-20, 
#                    display_convergence_message=True,
#                    display_steps=True, 
#                    custom_optimizer_DICT=custom_optimizer_DICT)
# GG.get_env(100)
# GG.plot_convergence()
# plt.show()

In [ ]:
Hamilt.molecule.fci_energy